In [1]:
# make it run on py2 and py3
from __future__ import division, print_function

In [2]:
# stretch Jupyter coding blocks to fit screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>")) 
# if 100% it would fit the screen

# Import Packages

In [3]:
import requests
import bs4 as bs
import pandas as pd
import re

# Connect to a website

In [4]:
Job_Title = ["Data Scientist", "Software Engineer", "Data Analyst", "Data Engineer", "Statistician", "DBA/Database Engineer", "Research Scientist", "Product/Project Manager", "Business Analyst"]

In [5]:
Job_Title_url = [job.replace(' ', '+') for job in Job_Title]
Job_Title_url

['Data+Scientist',
 'Software+Engineer',
 'Data+Analyst',
 'Data+Engineer',
 'Statistician',
 'DBA/Database+Engineer',
 'Research+Scientist',
 'Product/Project+Manager',
 'Business+Analyst']

In [ ]:
# Get content
# https://www.payscale.com/research/US/Job=Data_Analyst/Salary
# https://www.payscale.com/rcsearch.aspx?CountryName=United%2BStates&str=Business+Analyst
url = "https://www.payscale.com/rcsearch.aspx?CountryName=United%2BStates&str=" + Job_Title_url[5]
source = requests.get(url)

In [6]:
url = 'https://www.payscale.com/rcsearch.aspx?CountryName=United%2BStates&str=Business+Analyst'
source = requests.get(url)

In [7]:
# Transform to soup type
soup = bs.BeautifulSoup(source.content, features='html.parser') 

In [ ]:
if soup.find('div', class_='job__description') == None:
    candidate_block = soup.find_all('div', class_='RCSearchCategoryResultsContainer')
    Candidate_Jobs = [block for block in candidate_block if "Top Results for Job" in str(block)]
    url = 'https://www.payscale.com' + Candidate_Jobs[0].find_all('li', class_='RCSearchCategoryResultListItem')[0].find('a')['href']

    
    source = requests.get(url)
    soup = bs.BeautifulSoup(source.content, features='html.parser') 

# Locate to specific class

In [ ]:
Job_link = soup.find('link', rel='canonical')['href']

In [ ]:
AvgSalary = soup.find('div', class_='job__description').text
AvgSalary = re.findall(r"\$\d+\,?\d*", AvgSalary)[0]

In [ ]:
Vertical_Box = soup.find_all(class_='popular-skill__name')

In [ ]:
JobInformation = pd.DataFrame(columns = ['Job_Title', 'Salary', 'Skills', 'Href'])
JobInformation

In [ ]:
Vertical_Box[0]

In [ ]:
for block in Vertical_Box:
    Href = Job_link
    Job_Name = Job_Title[0]
    Salary = AvgSalary
    Skill = block.text
    JobList = [Job_Name, Salary, Skill, Href]
    JobList = pd.Series(JobList, index = JobInformation.columns)
    JobInformation = JobInformation.append(JobList, ignore_index=True)
JobInformation

In [ ]:
dir_path = os.path.dirname(os.path.realpath(__file__))
JobInformation.to_csv(os.path.join(dir_path, "JobInformation.csv"))